In [1]:
%load_ext autoreload
%autoreload 2

from math import pi
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

from root_pandas import read_root

Welcome to JupyROOT 6.14/00


In [2]:
import ROOT

In [3]:
!mkdir -p /home/nick/Dropbox/MicroBooNE/_fig/18_11_01/stop_muons/

In [4]:
save_folder = '/home/nick/Dropbox/MicroBooNE/_fig/18_11_01/stop_muons/'

In [5]:
# detector_x = [0, 256.35]
detector_x = [-50, 315]
detector_y = [-116.5, 116.5]
detector_z = [0, 1036.8]

In [ ]:
plt.rcParams.update({'font.size': 14})

## Useful functions

In [6]:
def point_is_fiducial(x, y, z, fiducial_x=[10, -10], fiducial_y=[30, -30], fiducial_z=[30, -30]):
#     is_x = (detector_x[0] + fiducial_x[0] < df[x]) & (df[x] < detector_x[1] + fiducial_x[1])
    is_y = (detector_y[0] + fiducial_y[0] < df[y]) & (df[y] < detector_y[1] + fiducial_y[1])
    is_z = (detector_z[0] + fiducial_z[0] < df[z]) & (df[z] < detector_z[1] + fiducial_z[1])
    return is_y & is_z

def PitchY(xs,ys,zs,xe,ye,ze):
    sdir = np.array([df[xs],df[ys],df[zs]])
    edir = np.array([df[xe],df[ye],df[ze]])
    tdir = edir-sdir
    tmag = np.sqrt(tdir[0]**2+tdir[1]**2+tdir[2]**2)
    theta = np.arccos(tdir[2]/tmag)
    #theta = np.arccos(tdir.Z())
    return theta

def MediandQdx(x):
    return np.median(x['_dqdx_v'])

In [7]:
def plot_stop_through_going(df, variable, binning, cut=10):
    stop_muons = df.query('0 < _yz_true_reco_distance < {}'.format(cut))[variable].values
    through_muons = df.query('_yz_true_reco_distance > {}'.format(cut))[variable].values
    plt.hist(through_muons, 
             bins=binning[0], 
             range=(binning[1], binning[2]), 
             label='through muons', 
             density=True, 
             histtype='step',
             lw=2)
    plt.hist(stop_muons, 
             bins=binning[0], 
             range=(binning[1], binning[2]), 
             label='stop muons', 
             density=True, 
             histtype='step',
             lw=2)
    plt.legend()
    plt.xlabel(variable)
    plt.ylabel('entries / {:.2f}'.format((binning[2]-binning[1])/binning[0]))

## Load datasets

In [8]:
# file_dir = "/home/david/data/StopMuCalibration/"
file_dir = "/home/nick/Desktop/MicroBooNE/StopMuCalibration/v07_07_03/stopmu_tag2/1030/prodgenie_bnb_nu_cosmic_uboone_mcc9.0_beta2_oct_reco_2d_wc_reco2/v1/*"
file_list = glob(file_dir+"/stopmu_ana.root")
# print(file_list)

In [9]:
df = read_root(file_list, key="stopmu/reco_tree")
print(df.shape)

(942606, 21)


In [11]:
df[['_trk_len', '_trk_start_x', '_trk_start_y', '_trk_start_z',
       '_trk_end_x', '_trk_end_y', '_trk_end_z', '_yz_true_reco_distance', '_delta_t_closest_flash'
]]

,_trk_len,_trk_start_x,_trk_start_y,_trk_start_z,_trk_end_x,_trk_end_y,_trk_end_z,_yz_true_reco_distance,_delta_t_closest_flash
0,415.073578,256.708649,101.018806,644.223328,150.865173,-101.021042,299.762238,5.554678,-50.500156
1,278.820801,248.215469,114.659935,510.790771,176.154434,-115.618835,647.672241,186.152237,7.420993
2,237.374786,67.040718,109.313835,601.035706,131.868530,-101.825935,514.686340,216.415466,-95.368538
3,244.894073,153.668655,112.610123,442.982880,69.683250,-115.039749,410.515625,112.530037,-21.742731
4,252.626434,46.699718,114.447701,242.403259,-36.014652,-107.742477,156.645645,77.345558,7.349312
5,217.801956,185.626892,103.110580,427.074280,177.782898,-101.956856,358.009491,59.854603,-17.054590
6,214.007858,231.290344,91.241295,246.051376,197.596222,-112.502625,190.234390,96.529312,11.613611
7,228.993301,119.192139,113.853333,274.658356,112.480499,-111.296448,314.935730,17.327486,14.246150
8,212.276001,194.523331,90.477852,235.384262,186.113647,-99.454430,298.085358,6.934430,-11.305313
9,158.739838,78.509918,110.935272,980.349121,36.212494,-32.848389,1032.332642,74.309471,2.022179


In [ ]:
selection = '0<_yz_true_reco_distance<10 & _trk_len > 50'
dqdx_u = np.hstack(df.query(selection)['_dqdx_u'].values)
dqdx_v = np.hstack(df.query(selection)['_dqdx_v'].values)
dqdx_y = np.hstack(df.query(selection)['_dqdx_y'].values)

rr_u = np.hstack(df.query(selection)['_rr_u'].values)
rr_v = np.hstack(df.query(selection)['_rr_v'].values)
rr_y = np.hstack(df.query(selection)['_rr_y'].values)

plt.hist2d(x=rr_u, y=dqdx_u, bins=(200, 200), range=((0,200), (0, 600)));
plt.title('dQdx vs residual range, plane U')
plt.xlabel('Residual range [cm]')
plt.ylabel('dQ/dx [adc/cm]')
plt.tight_layout()
plt.savefig(save_folder+'dqdx_u_only_stop_truth.png', dpi=500)
plt.close()

plt.hist2d(x=rr_v, y=dqdx_v, bins=(200, 200), range=((0,200), (0, 600)));
plt.title('dQdx vs residual range, plane V')
plt.xlabel('Residual range [cm]')
plt.ylabel('dQ/dx [adc/cm]')
plt.tight_layout()
plt.savefig(save_folder+'dqdx_v_only_stop_truth.png', dpi=500)
plt.close()

plt.hist2d(x=rr_y, y=dqdx_y, bins=(200, 200), range=((0,200), (0, 600)));
plt.title('dQdx vs residual range, plane Y')
plt.xlabel('Residual range [cm]')
plt.ylabel('dQ/dx [adc/cm]')
plt.tight_layout()
plt.savefig(save_folder+'dqdx_y_only_stop_truth.png', dpi=500)
plt.close()

In [ ]:
df['start_is_fiducial'] = point_is_fiducial('_trk_start_x', '_trk_start_y', '_trk_start_z')
df['end_is_fiducial'] = point_is_fiducial('_trk_end_x', '_trk_end_y', '_trk_end_z')
df['start_x_fiducial'] = (detector_x[0] + 15 < df['_trk_start_x']) & (df['_trk_start_x'] < detector_x[1] -15)
df['end_x_fiducial'] = (detector_x[0] + 15 < df['_trk_end_x']) & (df['_trk_end_x'] < detector_x[1] -15)
df['far_from_bottom'] = df['_trk_end_y'] > -90
df['start_end_y'] = df['_trk_start_y'] - df['_trk_end_y']

In [ ]:
cuts = [
    '_trk_len>150',
    'far_from_bottom==True',
    '(_trk_start_y >_trk_end_y)',
    '(30 < _trk_end_z < 1000)',
    '(abs(_delta_t_closest_flash) > 10)',
#     'start_x_fiducial==True',
    'end_x_fiducial==True'
]
selection = " & ".join(cuts)
selection_through = " & ".join(cuts + ['_yz_true_reco_distance>10'])
selection_stop = " & ".join(cuts + ['0<_yz_true_reco_distance<10'])

plt.subplot(1, 2, 1)
plt.hist2d(x=df.query(selection_through)['_trk_start_y'],
           y=df.query(selection_through)['_trk_end_y'],
           bins=(100,100),
           range=([-120,120],[-120,120]),
           norm=colors.LogNorm());
plt.colorbar()
plt.xlabel('start y [cm]')
plt.ylabel('end y [cm]')
plt.title("through going")

plt.subplot(1, 2, 2)
plt.hist2d(x=df.query(selection_stop)['_trk_start_y'],
           y=df.query(selection_stop)['_trk_end_y'],
           bins=(100,100),
           range=([-120,120],[-120,120]),
           norm=colors.LogNorm());
plt.colorbar()
plt.xlabel('start y [cm]')
# plt.ylabel('end y [cm]')
plt.title("stop")
plt.tight_layout()

plt.savefig(save_folder+'2dxy_'+selection+'.png', dpi=500)

In [ ]:
df['_yz_true_reco_distance'].hist(bins=105, range=(-5,100))
plt.xlabel('yz distance [cm]')
plt.ylabel('Entries / 1 cm')
# plt.title('Distance in the yz plane between reco and true and point')
plt.tight_layout()
# plt.savefig(save_folder+'yz_distance.png', dpi=500)

In [ ]:
variables_binning = {
#     '_delta_t_closest_flash': [200, -100, 100],
#     '_trk_start_x': [100, -60, 340],
#     '_trk_end_x': [100, -60, 340],
#     '_trk_start_y': [100, -120, +120],
#     '_trk_end_y': [100, -120, +120],
#     '_trk_start_z': [200, 0, 1100],
#     '_trk_end_z': [200, 0, 1100],
    'start_end_y': [200, -20, 20],
    '_trk_len': [200, 0, 100],
}

In [ ]:
for var, binning in variables_binning.items():
    plot_stop_through_going(df, var, binning=binning, cut=10)   
    plt.tight_layout()
    plt.savefig(save_folder+var+'.png', dpi=500)
    plt.close()

In [ ]:
cuts = [
    '0<_yz_true_reco_distance < 10',
    '_trk_len<200',
#     'far_from_bottom==True',
#     '(_trk_start_y >_trk_end_y)',
#     '(30 < _trk_end_z < 1000)',
#     '(abs(_delta_t_closest_flash) > 10)',
#     'end_x_fiducial==True'
]
selection = ' & '.join(cuts)
selected_df = df.query(selection)
print(selected_df.shape)

In [ ]:
dqdx_u = np.hstack(selected_df['_dqdx_u'].values)
dqdx_v = np.hstack(selected_df['_dqdx_v'].values)
dqdx_y = np.hstack(selected_df['_dqdx_y'].values)

rr_u = np.hstack(selected_df['_rr_u'].values)
rr_v = np.hstack(selected_df['_rr_v'].values)
rr_y = np.hstack(selected_df['_rr_y'].values)

plt.hist2d(x=rr_u, y=dqdx_u, bins=(200, 200), range=((0,200), (0, 600)));
plt.title('dQdx vs residual range, plane U')
plt.xlabel('Residual range [cm]')
plt.ylabel('dQ/dx [adc/cm]')
plt.tight_layout()
plt.savefig(save_folder+'dqdx_u_selected.png', dpi=500)
plt.close()

plt.hist2d(x=rr_v, y=dqdx_v, bins=(200, 200), range=((0,200), (0, 600)));
plt.title('dQdx vs residual range, plane V')
plt.xlabel('Residual range [cm]')
plt.ylabel('dQ/dx [adc/cm]')
plt.tight_layout()
plt.savefig(save_folder+'dqdx_v_selected.png', dpi=500)
plt.close()

plt.hist2d(x=rr_y, y=dqdx_y, bins=(200, 200), range=((0,200), (0, 600)));
plt.title('dQdx vs residual range, plane Y')
plt.xlabel('Residual range [cm]')
plt.ylabel('dQ/dx [adc/cm]')
plt.tight_layout()
plt.savefig(save_folder+'dqdx_y_selected.png', dpi=500)
plt.close()

In [ ]:
for var, binning in variables_binning.items():
    plot_stop_through_going(selected_df, var, binning=binning, cut=10)   
    plt.tight_layout()
    plt.savefig(save_folder+var+'_selected.png', dpi=500)
    plt.close()

In [ ]:
selected_df['_yz_true_reco_distance'].hist(bins=20, range=(-0.), log=True)
plt.xlabel('yz distance [cm]')
plt.ylabel('Entries / 1 cm')
# plt.title('Distance in the yz plane between reco and true and point')
plt.tight_layout()
plt.savefig(save_folder+'yz_distance_selected.png', dpi=500)

In [ ]:
dqdx_u = np.hstack(df['_dqdx_u'].values)
dqdx_v = np.hstack(df['_dqdx_v'].values)
dqdx_y = np.hstack(df['_dqdx_y'].values)

rr_u = np.hstack(df['_rr_u'].values)
rr_v = np.hstack(df['_rr_v'].values)
rr_y = np.hstack(df['_rr_y'].values)

plt.hist2d(x=rr_u, y=dqdx_u, bins=(30, 30), range=((0,200), (0, 600)));
plt.title('dQdx vs residual range, plane U')

plt.hist2d(x=rr_v, y=dqdx_v, bins=(30, 30), range=((0,200), (0, 600)));
plt.title('dQdx vs residual range, plane V')

plt.hist2d(x=rr_y, y=dqdx_y, bins=(30, 30), range=((0,200), (0, 600)));
plt.title('dQdx vs residual range, plane Y')

In [ ]:
df.query()['_yz_true_reco_distance'].hist(bins=100, range=(0,100))

In [ ]:
df['_yz_true_reco_distance'].hist(bins=100, range=(0,100))

In [ ]:
df['theta'] = PitchY('_trk_start_x','_trk_start_y','_trk_start_z','_trk_end_x','_trk_end_y','_trk_end_z')

In [ ]:
df['pitch'] = 0.3/np.cos(df['theta'])#df.apply(lambda x : 0.3/np.cos(df['theta']),axis=1)

In [ ]:
#df['median_dqdx'] = df.apply(lambda x: MediandQdx(x), axis=1)

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.hist(np.abs(df['pitch'].values),bins=np.linspace(0.3,1,100), density=True)
plt.grid()
plt.xlabel("pitch [cm]")
plt.ylabel("Area normalized / {:.2f} cm".format((1-0.3)/100))
plt.title("pitch Y plane, tag0")
plt.show()

In [ ]:
from Recombination import RecombModBox
from LandauEloss import dedx, dpdx

In [ ]:
selected2_df = df.query('start_is_fiducial==False & end_is_fiducial==False & _trk_len>50 and _pl==2')
selected1_df = df.query('start_is_fiducial==False & end_is_fiducial==False & _trk_len>50 and _pl==1')
selected0_df = df.query('start_is_fiducial==False & end_is_fiducial==False & _trk_len>50 and _pl==0')

In [ ]:
dqdx_v_2 = np.hstack(selected2_df['_dqdx_v'].values)
dqdx_v_1 = np.hstack(selected1_df['_dqdx_v'].values)
dqdx_v_0 = np.hstack(selected0_df['_dqdx_v'].values)

In [ ]:
fig = plt.figure(figsize=(6,6))
BINS = np.linspace(0,400,100)
plt.hist(dqdx_v_2,bins=BINS,color='r',lw=2,histtype='step',label='Pl 2')
plt.hist(dqdx_v_1,bins=BINS,color='g',lw=2,histtype='step',label='Pl 1')
plt.hist(dqdx_v_0,bins=BINS,color='b',lw=2,histtype='step',label='Pl 0')
plt.grid()
plt.xlabel('dQ/dx [ADC/cm]')
plt.gca().get_yaxis().set_visible(False)
plt.legend()
plt.show()

In [ ]:
# assume uniform distribution from 1 to 10 GeV
# sample from pitch distribution of sample

dpdx_v_th = []
dqdx_v_th = []

for i in xrange(60000):
    mu_energy = np.random.rand()*9 + 1
    pitch = np.abs(df['pitch'][i])
    dpdxval = dpdx(mu_energy*1e3,pitch,105.)
    R = RecombModBox(dpdxval)
    dqdx = dpdxval * (1-R) / (23.6/1e3)
    dpdx_v_th.append(dpdxval)
    dqdx_v_th.append(dqdx)

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.hist(dpdx_v_th,bins=np.linspace(1,3,100))
plt.grid()
plt.title('expected dE/dx from $\mu$ flux')
plt.xlabel('dE/dx [MeV/cm]')
plt.gca().get_yaxis().set_visible(False)
plt.show()

fig = plt.figure(figsize=(6,6))
plt.hist(dqdx_v_th,bins=np.linspace(4e1,6e1,100))
med = np.median(dqdx_v_th)
std = np.std(dqdx_v_th)
plt.errorbar(med,1000.,xerr=std,fmt='o',color='r',lw=2,label='%.0f $\pm$ %.0f'%(med,std))
plt.grid()
plt.legend()
plt.title('expected dQ/dx from $\mu$ flux')
plt.xlabel('dQ/dx [1k $e^-$ /cm]')
plt.gca().get_yaxis().set_visible(False)
plt.show()

In [ ]:
# pylandau from https://github.com/SiLab-Bonn/pylandau
from pylandau import langau

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# PLANE 2
vals2,bine2 = np.histogram(dqdx_v_2,bins=np.linspace(100,400,100))
valsmax2 = np.max(vals2)
vals2 = vals2.astype(float)
vals2 /= valsmax2
binc2 = 0.5*(bine2[1:]+bine2[:-1])
# PLANE 1
vals1,bine1 = np.histogram(dqdx_v_1,bins=np.linspace(100,400,100))
valsmax1 = np.max(vals1)
vals1 = vals1.astype(float)
vals1 /= valsmax1
binc1 = 0.5*(bine1[1:]+bine1[:-1])
# PLANE 0
vals0,bine0 = np.histogram(dqdx_v_0,bins=np.linspace(100,400,100))
valsmax0 = np.max(vals0)
vals0 = vals0.astype(float)
vals0 /= valsmax0
binc0 = 0.5*(bine0[1:]+bine0[:-1])

In [ ]:
guess2 = [200,20,20]
guess1 = [200,40,40]
guess0 = [200,40,40]
popt2,popv2 = curve_fit(langau,binc2,vals2,p0=guess2)
popt1,popv1 = curve_fit(langau,binc1,vals1,p0=guess1)
popt0,popv0 = curve_fit(langau,binc0,vals0,p0=guess0)

In [ ]:
print(popt2)
print(popt1)
print(popt0)

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.plot(binc2,vals2,'r-',lw=2,label='Pl 2: MPV = %.0f $\sigma$ = %.0f'%(popt2[0],popt2[2]))
plt.plot(binc2,langau(binc2,*popt2),'r--',lw=3)
plt.plot(binc1,vals1,'g-',lw=2,label='Pl 1: MPV = %.0f $\sigma$ = %.0f'%(popt1[0],popt1[2]))
plt.plot(binc1,langau(binc1,*popt1),'g--',lw=3)
plt.plot(binc0,vals0,'b-',lw=2,label='Pl 0: MPV = %.0f $\sigma$ = %.0f'%(popt0[0],popt0[2]))
plt.plot(binc0,langau(binc0,*popt0),'b--',lw=3)
plt.grid()
plt.xlabel('dQ/dx [ADC/cm]')
plt.legend(loc=1)
plt.show()

In [ ]:
# Gain obtained by tuning MPV to theoretical expectation of 50 +/- 2
G2  = 5e4 / popt2[0]
G2e = 2e3 / popt2[0]
G1  = 5e4 / popt1[0]
G1e = 2e3 / popt1[0]
G0  = 5e4 / popt0[0]
G0e = 2e3 / popt0[0]

In [ ]:
print('Gain Pl2 : %.01f +/- %.01f e-/ADC'%(G2,G2e))
print('Gain Pl1 : %.01f +/- %.01f e-/ADC'%(G1,G1e))
print('Gain Pl0 : %.01f +/- %.01f e-/ADC'%(G0,G0e))

In [ ]:
fig = plt.figure(figsize=(10,6))
plt.plot(binc2,vals2,'r-',lw=2,label='Pl 2: Gain = %.0f $\pm$ %.0f  e-/ADC'%(G2,G2e))
plt.plot(binc2,langau(binc2,*popt2),'r--',lw=3)
plt.plot(binc1,vals1,'g-',lw=2,label='Pl 1: Gain = %.0f $\pm$ %.0f  e-/ADC'%(G1,G1e))
plt.plot(binc1,langau(binc1,*popt1),'g--',lw=3)
plt.plot(binc0,vals0,'b-',lw=2,label='Pl 0: Gain = %.0f $\pm$ %.0f  e-/ADC'%(G0,G0e))
plt.plot(binc0,langau(binc0,*popt0),'b--',lw=3)
plt.grid()
plt.xlabel('dQ/dx [ADC/cm]')
plt.legend(loc=1,fontsize=16)
plt.show()